In [7]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

# 상수 정의 (이미지 경로)
IMAGE_PATH = 'data/test/0a0302cbe2dcdd40.jpg'
TRAIN_DATA_DIR = 'data/train_kr_class/'


In [8]:
# 1. EfficientNet-B4 모델 정의
class RotationPredictor(nn.Module):
    def __init__(self):
        super(RotationPredictor, self).__init__()
        # torchvision 모델로부터 사전 학습된 EfficientNet-B4 불러오기
        self.efficientnet = models.efficientnet_b4(pretrained=True)
        # 마지막 레이어 수정 (회전 각도를 예측하기 위해)
        self.efficientnet.classifier[1] = nn.Linear(self.efficientnet.classifier[1].in_features, 1)
    
    def forward(self, x):
        return self.efficientnet(x)

# 모델 인스턴스 생성
model = RotationPredictor()

# 2. 이미지 전처리 및 후처리 함수 정의
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def postprocess(image_tensor):
    image_tensor = image_tensor.squeeze(0)
    image_tensor = image_tensor.permute(1, 2, 0)
    image_tensor = image_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
    image_tensor = image_tensor.numpy()
    image_tensor = np.clip(image_tensor, 0, 1)
    return image_tensor

# 3. 학습용 데이터셋 및 데이터로더 설정
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(45),  # 랜덤 회전을 통해 데이터 증강
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(TRAIN_DATA_DIR, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 4. 학습 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 모델 학습 함수 정의
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for inputs, _ in train_loader:  # 데이터와 라벨(라벨은 필요 없음)을 불러옴
            optimizer.zero_grad()
            outputs = model(inputs)
            
            # 회전 각도에 대한 타겟을 생성 (여기선 랜덤 타겟을 생성)
            # 실제 사용 시 각 이미지의 회전 각도를 라벨로 사용해야 함
            target_angles = torch.randn(outputs.size()).to(outputs.device)
            
            loss = criterion(outputs, target_angles)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# 6. 모델 학습 실행
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

# 7. 회전 보정 함수 정의 및 예시 이미지 로드
def correct_rotation(image, model):
    # 이미지 전처리
    img_input = preprocess(image).unsqueeze(0)
    
    # 모델을 평가 모드로 전환
    model.eval()
    
    with torch.no_grad():
        # 회전 각도 예측
        predicted_angle = model(img_input).item()
    
    # 이미지 회전 보정
    corrected_image = image.rotate(-predicted_angle)
    
    return corrected_image, predicted_angle

image = Image.open(IMAGE_PATH).convert('RGB')

# 회전 보정
corrected_image, predicted_angle = correct_rotation(image, model)

# 8. 원본 이미지와 회전 보정된 이미지 시각화
plt.figure(figsize=(12, 6))

# 원본 이미지
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original Rotated Image")
plt.axis('off')

# 회전 보정된 이미지
plt.subplot(1, 2, 2)
plt.imshow(corrected_image)
plt.title(f"Corrected Image\n(Predicted Angle: {predicted_angle:.2f}°)")
plt.axis('off')

plt.show()

FileNotFoundError: Couldn't find any class folder in data/train/.